In [1]:
%matplotlib nbagg
import numpy as np
import matplotlib.pyplot as plt
import time
import os
os.environ["CUDA_VISIBLE_DEVICES"]="0"
import tensorflow as tf
import pickle

import tensorflow.keras.backend as K
from model import TBPP384
from tbpp_prior import PriorUtil
from tbpp_data import InputGenerator
from tbpp_training import TBPPFocalLoss
from utils.model import load_weights
from utils.training import Logger

### Check using devices

In [2]:
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 4907133632768135694
, name: "/device:XLA_CPU:0"
device_type: "XLA_CPU"
memory_limit: 17179869184
locality {
}
incarnation: 2518925033299319451
physical_device_desc: "device: XLA_CPU device"
, name: "/device:XLA_GPU:0"
device_type: "XLA_GPU"
memory_limit: 17179869184
locality {
}
incarnation: 15739487107650835827
physical_device_desc: "device: XLA_GPU device"
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 15658998170
locality {
  bus_id: 1
  links {
  }
}
incarnation: 9139163663857341384
physical_device_desc: "device: 0, name: Tesla V100-PCIE-16GB, pci bus id: 3e94:00:00.0, compute capability: 7.0"
]


### Data

In [3]:
from data_synthtext import GTUtility
gt_util = GTUtility('data/SynthText/', polygon=True)
gt_util_train, gt_util_val = gt_util.split(0.9)

### Data check

In [ ]:
idx = 4
_, image, label = gt_util_train.sample()

In [ ]:
plt.imshow(image)
gt_util_train.plot_gt(label,show_labels=False)
plt.show()

In [ ]:
image.shape

In [ ]:
label[:,6] * 600

In [ ]:
plt.imshow(image)
plt.show()

### Model

In [4]:
# TextBoxes++
K.clear_session()
model = TBPP384(softmax=False)
weights_path = None

experiment = 'tbpp384fl_synthtext'

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


In [5]:
prior_util = PriorUtil(model)

### Saving parameters

In [6]:
import json

* Add custom Layer

In [7]:
from tensorflow.keras.utils import get_custom_objects
from utils.layers import Normalize

In [8]:
get_custom_objects().update({
    "Normalize": Normalize
})

In [9]:
with open("../model_params.json",'w') as f:
    f.write(model.to_json())

In [10]:
from tensorflow.keras.models import model_from_json

In [11]:
with open("../model_params.json",'r') as f:
    parameter_saved_model = model_from_json(f.read())

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


### Training

In [ ]:
# os.environ['TF_ENABLE_AUTO_MIXED_PRECISION'] = '1'

* checking available batch size of validation set

```
gen_val = InputGenerator(
    gt_util_val, prior_util, batch_size*4, model.image_size)
```python

In [12]:
from multiprocessing import cpu_count
from tensorflow.keras.callbacks import ModelCheckpoint

In [13]:
epochs = 10
initial_epoch = 0
batch_size = 32

gen_train = InputGenerator(gt_util_train, prior_util, batch_size, model.image_size)
gen_val = InputGenerator(gt_util_val, prior_util, batch_size*4, model.image_size)

checkdir = './checkpoints/' + time.strftime('%Y%m%d%H%M') + '_' + experiment
if not os.path.exists(checkdir):
    os.makedirs(checkdir)

with open(checkdir+'/source.py','wb') as f:
    source = ''.join(['# In[%i]\n%s\n\n' % (i, In[i]) for i in range(len(In))])
    f.write(source.encode())

optim = tf.keras.optimizers.SGD(lr=1e-3, momentum=0.9, decay=0, nesterov=True)
# optim = tf.keras.optimizers.Adam(lr=1e-3, beta_1=0.9, beta_2=0.999, epsilon=0.001, decay=0.0)

# weight decay
regularizer = tf.keras.regularizers.l2(5e-4) # None if disabled
#regularizer = None
for l in model.layers:
    if l.__class__.__name__.startswith('Conv'):
        l.kernel_regularizer = regularizer

loss = TBPPFocalLoss(lambda_conf=10000.0, lambda_offsets=1.0)

model.compile(optimizer=optim, loss=loss.compute, metrics=loss.metrics)

callbacks = [
    ModelCheckpoint(checkdir+'/weights.{epoch:03d}.h5', 
                    verbose=1, save_weights_only=True),
    Logger(checkdir)
]

print(checkdir.split('/')[-1])
history = model.fit_generator(
        gen_train.generate(),
        epochs=epochs, 
        steps_per_epoch=int(gen_train.num_batches/4), 
        callbacks=callbacks,
        validation_data=gen_val.generate(), 
        validation_steps=int(gen_val.num_batches/4),
        workers=cpu_count(), 
        use_multiprocessing=True)

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
201911120606_tbpp384fl_synthtext
Epoch 1/10
6037/6038 [============================>.] - ETA: 0s - loss: 44.1309WARNING:tensorflow:Using a generator with `use_multiprocessing=True` and multiple workers may duplicate your data. Please consider using the `keras.utils.Sequence` class.

Epoch 00001: saving model to ./checkpoints/201911120606_tbpp384fl_synthtext/weights.001.h5
6038/6038 [==============================] - 4055s 672ms/step - loss: 44.1298 - val_loss: 38.6682
Epoch 2/10
6037/6038 [============================>.] - ETA: 0s - loss: 31.1446WARNING:tensorflow:Using a generator with `use_multiprocessing=True` and multiple workers may duplicate your data. Please consider using the `keras.utils.Sequence` class.

Epoch 00002: saving model to ./checkpoints/201911120606_tbpp384fl_synthtext/weights.002.h5
6038/6038 [==============================] - 4021s 666ms/step - loss: 31.1441 - val_loss: 2